# Allianz Pricing: From Data Preparation to Risk Modeling (Data Preparation Part)

## 1. Data preparation

**[Insert Context for Task 1]**

In this task, we will discuss the need of data in the insurance context and how to practically prepare it for statistical analysis. Also, we will do some basic but practical common checks on claims and policy data.

## Good to know

This project assumes you have a solid foundation in Python, statistics, and machine learning. Before attempting this project, it is recommended that you complete the prerequisite courses.

**[Insert Instructions Below]**
- Before merging the policy data and claim data, check if the `IDpol` duplicates in both data sets. If yes, aggregate separate policies whose `IDpol` are same into one single policy.
- Re-define the columns `IDpol` and `ClaimNb` as `int`. 
- Merge the policy and claim data (merging key: `IDpol`). For policies that don't have `ClaimAmount`, fill them with 0.

### Sample Code

In [1]:
import pandas as pd
# create data dictionary 
data_dic = pd.DataFrame.from_dict({
    'IDpol': 'policy number (unique identifier)',
    'ClaimNb': 'number of claims on the given policy',
    'Exposure': 'total exposure in yearly units',
    'Area': 'area code (categorical, ordinal)',
    'VehPower': 'power of the car (categorical, ordinal)',
    'VehAge': 'age of the car in years',
    'DrivAge': 'age of the (most common) driver in years',
    'BonusMalus': 'bonus-malus level between 50 and 230 (with base level 100)',
    'VehBrand': 'car brand (categorical, nominal)',
    'VehGas': 'diesel or regular fuel car (binary)',
    'Density': 'density of inhabitants per km square in the city of the living place of the driver',
    'Region': 'regions',
    'ClaimAmount': 'claim amount on the given policy'}, 
    orient='index', columns=['definition'])

# load both claim and policy data
pol_db = pd.read_csv("datasets/policy_db.csv")
clm_db = pd.read_csv("datasets/claim_db.csv")

print("no duplicated policy ID in policy data") if (len(pol_db) == len(set(pol_db))) else print("Policy ID duplicates in policy data")

# aggregate claim amount from same policy
# aggregate separate policies whose `IDpol` are same into one single policy
clm_db = clm_db.groupby("IDpol").sum()

# merge freq & sev datasets

# Re-define the columns `IDpol` and `ClaimNb` as `int`.
pol_db["IDpol"]   = pol_db["IDpol"].astype(int)
pol_db["ClaimNb"] = pol_db["ClaimNb"].astype(int)
pol_db.set_index("IDpol", inplace=True)

pol_clm_db = pol_db.join(clm_db, how="left")
pol_clm_db["ClaimAmount"].fillna(0, inplace=True)
pol_clm_db.reset_index(inplace=True)

Policy ID duplicates in policy data


In [10]:
import pandas as pd

pol_db = pd.read_csv("datasets/policy_db.csv")
clm_db = pd.read_csv("datasets/claim_db.csv")

print("no duplicated policy ID in policy data") if (len(pol_db) == len(set(pol_db))) else print("Policy ID duplicates in policy data")

clm_db
clm_db = clm_db.groupby("IDpol").sum()
clm_db
pol_db["IDpol"]   = pol_db["IDpol"].astype(int)
pol_db["ClaimNb"] = pol_db["ClaimNb"].astype(int)

len(pol_db), len(clm_db)
pol_db.set_index("IDpol", inplace=True)

#pol_clm_db = pol_db.join(clm_db, how="left")

Policy ID duplicates in policy data


## 2. Inspect the dataset

**[Insert Context for Task 2]**

Get familiar with the new merged dataset.

**[Instructions]**

- Use pandas dataframe.info() method to have a initial review on data information. 
- Check whether the number of rows remain the same after the data merge. 
- Check whether missing value exists.

### Sample Code

In [3]:
pol_clm_db.info()
pol_clm_db.tail(17)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678013 entries, 0 to 678012
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   IDpol        678013 non-null  float64
 1   ClaimNb      678013 non-null  int32  
 2   Exposure     678013 non-null  float64
 3   Area         678013 non-null  object 
 4   VehPower     678013 non-null  float64
 5   VehAge       678013 non-null  float64
 6   DrivAge      678013 non-null  float64
 7   BonusMalus   678013 non-null  float64
 8   VehBrand     678013 non-null  object 
 9   VehGas       678013 non-null  object 
 10  Density      678013 non-null  float64
 11  Region       678013 non-null  object 
 12  ClaimAmount  678013 non-null  float64
dtypes: float64(8), int32(1), object(4)
memory usage: 64.7+ MB


## 3. Preliminary data overview

**[Replace with Context for Task 3]**

Get the basic KPI for the policy-claim data.

**[Instructions]**

- Group the data (`pol_clm_db`) with respect to the number of claims(`ClaimNb`) and calculate the total exposures (`Exposure`) and number of policies at each sub-group level of "number of claims (`ClaimNb`)".
- Calculate the overall portfolio frequency, i.e. total number of claims divided by total exposure.

### Sample Code

In [4]:
pol_clm_db.groupby("ClaimNb")[["ClaimNb", "Exposure"]].sum()
pol_clm_db["Frequency"] = pol_clm_db["ClaimNb"] / pol_clm_db["Exposure"]

## 4. Further analysis of the data

**[Insert Context for Task 4]**

Determine whether policy renewal has happened during the year.

**[Instructions]**


- For policy whose exposure is equal to 1 year, what is the percentage of such policy. __Please note that the data used in this project only covers one whole calendar year.__ That means, if a policy has been renewed in reality but in our data set it's recorded as two separate policy. We can find their sum of exposure is 1. We can use this fact to detect whether policy renewal has happened.
- Is the answer from (1) seems rather unusual? In practice, policies belonging to the same policy holder should share the same policy ID and is noted as new business or renewal. But we don't have such granular information in this dataset.
- Determine whether the data set follow the practice mentioned above that same policy holder uses the same policy ID. If a policy had been renewed and was indeed recorded as two separate ones in this dataset, then the sum of their exposure should be 1.

In [5]:
pol_clm_db.pivot_table(values = "Exposure", index = "IDpol")

,Exposure
IDpol,
1.0,0.10000
3.0,0.77000
5.0,0.75000
10.0,0.09000
11.0,0.84000
...,...
6114326.0,0.00274
6114327.0,0.00274
6114328.0,0.00274


In [6]:
(pol_clm_db.groupby("IDpol")["Exposure"].sum() )

IDpol
1.0          0.10000
3.0          0.77000
5.0          0.75000
10.0         0.09000
11.0         0.84000
              ...   
6114326.0    0.00274
6114327.0    0.00274
6114328.0    0.00274
6114329.0    0.00274
6114330.0    0.00274
Name: Exposure, Length: 678013, dtype: float64

**[Insert Hints Below]**

- groupby 'Area', 'VehPower', 'VehAge', 'DrivAge', 'BonusMalus', 'VehBrand', 'VehGas', 'Density', 'Region', sum the exposure and count the number for each sub-group level. It's highly likely that same policy holder should share the same features.

### Sample Code

In [ ]:
_______________________________


## 5. Exploratory data analysis

**[Context for Task 5]**

In this task, we will discuss how to modify the rating factors in order to make them usable for risk modeling. Also, we will introduce how to perform a univariate/multivariate analysis of each rating factor in terms of volumes and observed frequency/severity, which lay the ground for exploring dependence structure among feature components and for risk modeling as well.



**[Add instructions]**

- Plot empirical distribution for `Exposure` and `CliamNb`. Please also plot empirical distribution for those policies whose `Exposure` > 1 and `ClaimNb` > 4.

### Sample Code

In [ ]:

____________

## 6. Explanatory variables: duplicates (bonus | difficult)

**[Context for Task 6]**

Comment based on the plots from last question: 
- A significant number of policies have low Exposure, i.e. close to zero.
- Distribution of values is reasonable except that a spike (exposure = 0.08) occurs in `Exposure` plot (top left graph). We will further investigate it in the section below.


Let's further look into two potential data errors: 

1. Policies belonging to the same policy holder should normally share same policy ID and be aggregated; Otherwise, when modelling, it will violate the assumption of independent sample.

2. Duplicated policy: some policy is split into two parts where the sum of exposure equals the other policy(different `IDpol`) in this dataset.

we don't fix the error above since it requires more granular information and dedicated efforts, which is not the main aim of this project.

**[Add instructions]**

Find any example/policy that might belong to the same policy holder.

### Sample Code

In [ ]:
___________________

## 7. Abnormal behavior

**[Context for Task 7]**

As mentioned previously, there is a spike in exposure histogram where exposure = 0.08. Figure out potential reasons for this abnormal behavior.

**[Add Instructions]**

- Figure out potential reasons for this abnormal behavior.

### Sample Code

In [ ]:
pol_clm_db[pol_clm_db['Exposure']<1].hist(bins=100, column='Exposure')
_____________________

## 8. Data anomaly: Large claim number

**[Context for Task 8]**

The largest claim number is 16, which is unusual for this data set. Try to find reason to explain such anomaly.

**[Instructions]**

- No instruction

### Sample Code

In [ ]:

________________

## 9. Dealing with outliers

**[Context for Task 9]**

Some policy contains certainly outliers (large claim count), and look very suspiciously like either poor data or fraudulent activity. Given the fields in the input data, we'll never be able to fully ascertain the true cause. The options are:

1. Leave the data and hope any resulting model is unaffected. It may be a question of selecting a model specifically to be immune to this type of data.
2. Remove these suspicious records from the data.
3. Keep them but manually alter the data, e.g. cap the number of claims at 3 or 4, for example.

For the time being, we will cap the number of claims at 4 in modeling part. In fact, it'll be interesting to fit and assess any model with and without these extreme observations, to see what difference it makes.

**[Instructions]**

- Correct for those policy whose exposures are bigger than one year (by setting them equal to 1) since all observations are within one accounting year as per the data description. 
- It's easy to notice that 9 policies have more than 4 claims, the maximal number being 16. Please cap these values by 4 since these big values are likely to be data quote errors or fraud.

In [ ]:


_______________________


## 10. One-way analysis

**[Context for Task 10]**

A one-way analysis summarizes insurance statistics, such as frequency or loss ratio, for each value of each explanatory variable.

**[Instructions]**

- Read the function below for the one-way analysis, use it for variable "Area" and see what you can find. Also you can play it with other variables and try to extract business insights from the plots.

In [ ]:
def one_way_analysis(db, ow_var, expo_fr, expo_sev, response_fr, response_sev):
    
    _, axes = plt.subplots(nrows=2, ncols=2, figsize=(16,10))

    response_fr    = db.groupby(ow_var)[response_fr].sum()
    response_sev   = db.groupby(ow_var)[response_sev].sum()
    expo_fr        = db.groupby(ow_var)[expo_fr].sum()
    expo_sev       = db.groupby(ow_var)[expo_sev].sum()
    ratio_fr       = response_fr / expo_fr
    ratio_sev      = response_sev / expo_sev
    
    axes[0,0].title.set_text('Frequency')
    axes[0,1].title.set_text('Severity')
        
    ratio_fr.plot(rot=0, ax=axes[0,0], style='o--')
    expo_fr.plot.bar(rot=0, ax=axes[1,0])
    
    ratio_sev.plot(rot=0, ax=axes[0,1], style='o--')
    expo_sev.plot.bar(rot=0, ax=axes[1,1])


one_way_analysis(_______________)

one_way_analysis(_______________)

one_way_analysis(_______________)

## 11. Log transformation

**[Add Context for Task 11]**

Observation from one-way analysis:
1. The marginal empirical frequency is monotonically increasing for the area code, suggesting that the feature component Area is ordinal.
2. The marginal frequency is decreasing in vehicle age, and it has a "U" shape in driver's age (until age 50)
3. We see that the frequency is monotonically increasing in the log density of the population
4. By far the most explanatory feature component for frequency modeling is the bonus-malus level


Based on the observation above, we can do some simplification for the explainable features.

**[Update Instructions]**

- For variable `Area`, we found that, in frequency plot, the overall trend is upward. Please map the alphabet order into numerical order using dictionary `mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6}` 
- Capping variable `VehPower`, `VehAge`, `DrivAge` and `BonusMalus` at 9, 20, 90, 150, respectively, since the volume above these levels is negligible.
- Log transformation for variable `Density`

### Sample Code

In [ ]:
mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6} 
pol_clm_db['Area']          = _______________
pol_clm_db['VehPower']      = _______________
pol_clm_db['VehAge']        = _______________
pol_clm_db['DrivAge']       = _______________
pol_clm_db['BonusMalus']    = _______________
pol_clm_db['LogDensity']    = round(np.log(___________), 0)

## 12. Two-way analysis

**[Add Context for Task 12]**

We only do the numerical correlation in this case; It's also possible to get the correlation for categorical variables (e.g. Cramér's V)

**[Add Instructions for Task 12]**

- Caculate correlation value for numberical variables

### Sample Code

In [ ]:
pol_clm_db_corr = pol_clm_db.copy()

vars = ['Area', 'VehPower', 'VehAge', 'DrivAge', 'BonusMalus', 'Density'] #  'Area': catogory

____________

## 13. Large loss analysis

**[Context for Task 13]**

A large loss is a claim event that typically occurs with a low frequency but large severity. Some negative impact on the risk modeling from large loss could be:
- It's difficult to estimate large losses in the tail’s distribution between different years,
- It will bias the estimation because some specific segments can be largely influenced by large losses. 

In this section, we will discuss how to deal with large cost in order to model a more stable quantity.

**[Add Instructions]**

- Plot the empirical loss distribution for the whole data range and tail part. For tail threshold, one can choose certain value that he/she think it appropriate.

### Sample Code

In [ ]:
sev_db = pol_clm_db.loc[pol_clm_db['ClaimAmount']>0,['ClaimNb','ClaimAmount']]

_, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,5))

sev_db.hist(column='ClaimAmount', bins=100, log=True, ax=axes[0])
sev_db[sev_db['ClaimAmount']>_____].hist(column='ClaimAmount', bins=100, ax=axes[1])

axes[0].title.set_text('Claim Amount')
axes[1].title.set_text('Tail behavior: Claim Amount')

## 14. Large loss analysis: Identifying thresholds

**[Context for Task 14]**


**[Instructions]**

- Identify threshold for large claim. Please note there are lots of ways to find the threshold, one can choose whatever method he/she thinks appropriate.
- In this approach, we first get the percentage of claim amount over certain threshold, then we compute the percentage of claims number over the threshold. Finally, we define the metric, which is percentage of claim amount over threshold divided by percentage of claim number over threshold.

**[Insert Hints Below]**

- The approach here can be different as long as it can select a reasonable threshold.

### Sample Code

In [ ]:
________________________
________________________

### Solution Code

## 15. Plotting distributions

**[Add Context for Task 15]**

Visualize both attritional distribution and excess distribution.

**[Instructions]**

- Plot both attritional distribution and excess distribution given the large loss threshold equals 190,000

### Sample Code

In [ ]:
# attritional distribution
threshold = 190_000
_______________

# excess distribution
_______________